# PhishEye

### Import Statements

In [ ]:
# pip install dnstwist
# pip install DNSPython
# pip install pillow

In [3]:
import pandas as pd
import dnstwist
import numpy as np 
import matplotlib.pyplot as plt 


### Dataset Exploration

In [188]:
fuzz = dnstwist.Fuzzer("www.google.com")
fuzz.generate()
len(fuzz.permutations())

2158

In [189]:
data = dnstwist.run(domain='google.com', registered=True, format='null')
reg = [d['domain'] for d in data]


In [190]:
data_non = dnstwist.run(domain='google.com', unregistered=True, format='null')
nonreg = [d['domain'] for d in data_non]


In [ ]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

intersection(reg, nonreg)

In [ ]:
print (f'The number of registered permutations is: {len(reg)}')
print (f'The number of non registered permutations is: {len(nonreg)}')


In [ ]:
print (reg[::15])
print (nonreg[::150])

In [ ]:
domains_df = pd.read_csv('./top-1m.csv', header=None, index_col=0)
domains_df.head()

In [ ]:
def get_dicts(domain_list):
    reg_dict, nonreg_dict = {}, {}
    for domain in domain_list:
        data_reg = dnstwist.run(domain=f'{domain}', registered=True, format='null')
        reg = [d['domain'] for d in data_reg]
        reg_dict[domain]  = len(reg)
        data_nonreg = dnstwist.run(domain=f'{domain}', unregistered=True, format='null')
        nonreg = [d['domain'] for d in data_nonreg]
        nonreg_dict[domain]  = len(nonreg)
    return reg_dict, nonreg_dict
# eda_reg, eda_nonreg = get_dicts(list(domains_df[1].values[:10]))
# eda_reg


In [ ]:
# x = list(eda_reg.keys())
# Yreg = list(eda_reg.values())
# Znonreg= list(eda_nonreg.values())
# X_axis = np.arange(len(x))
  
# plt.bar(x, Znonreg, color='steelblue')
# plt.bar(x, Yreg, bottom=Znonreg, color='darkorange')
  
# plt.xlabel("Domains")
# plt.ylabel("Number of Permutations")
# plt.title("Number of Registered and Non Registered Domain Permutations")
# plt.xticks(rotation=30)

# plt.legend(labels = ['Non Registered: Benign', 'Registered: Malicious'])
# plt.show()

In [ ]:
def create_twist_dict(domains):
    twist_dict = {}
    for domain in domains:
        #twist_dict[domain] = [[],[]]
        data_reg = dnstwist.run(domain=f'{domain}', registered=True, format='null')
        reg = [d['domain'] for d in data_reg]
        for homograph in reg:
            twist_dict[homograph] = [domain, True]

        data_nonreg = dnstwist.run(domain=f'{domain}', unregistered=True, format='null')
        nonreg = [d['domain'] for d in data_nonreg]
        for homograph in nonreg:
            twist_dict[homograph] = [domain, False]
    return twist_dict

twisted_dict = create_twist_dict(list(domains_df[1].values[:10]))

In [ ]:
twisted_df = pd.DataFrame.from_dict(twisted_dict, orient='index').reset_index()
twisted_df.columns = ['Homograph', 'Domain', 'Registered']
twisted_df.to_csv('twisted.csv')
twisted_df


### Text Distance Metrics

In [133]:
from strsimpy.levenshtein import Levenshtein
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.sorensen_dice import SorensenDice
from strsimpy.cosine import Cosine

test_string1 = reg[0] # google.com
test_string2 = nonreg[1] # g00qle.com

levenshtein = Levenshtein()
print(levenshtein.distance(test_string1, test_string2))

jarowinkler = JaroWinkler()
print(jarowinkler.distance(test_string1, test_string2))

sorensondice = SorensenDice()
print(sorensondice.distance(test_string1, test_string2))

cosine = Cosine(2)
a = cosine.get_profile(test_string1)
b = cosine.get_profile(test_string2)
print(cosine.similarity_profiles(a,b))


NameError: name 'reg' is not defined

In [ ]:
for index, row in twisted_df.iterrows():
    twisted_df.loc[index,'Levenshtein'] = levenshtein.distance(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'Jaro-Winkler'] = jarowinkler.distance(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'Sorenson-Dice'] = sorensondice.distance(row['Domain'], row['Homograph'])
    str_to_vect_a= cosine.get_profile(row['Domain'])
    str_to_vect_b= cosine.get_profile(row['Homograph'])
    twisted_df.loc[index,'Cosine'] = cosine.similarity_profiles(str_to_vect_a, str_to_vect_b)

twisted_df    

In [103]:
# twisted_df.to_csv('twisted_text_distance.csv')
twisted_df = pd.read_csv('twisted_text_distance.csv')

### Image Similarity

In [8]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
# test sizing
lengths = [len(s) for s in twisted_df.Homograph]
longest_idx= lengths.index(max(lengths))
text = twisted_df.Homograph[longest_idx]
img = Image.new('RGB', (1024, 128))
# use bold font
font = ImageFont.truetype(f"./fonts/arial bold.ttf",70)
# draw image
d1 = ImageDraw.Draw(img)
# Center text in image
xpos = (img.size[0] / 2) - (font.getsize(text)[0]/2)
ypos = (img.size[1] / 2) - (font.getsize(text)[1]/2)
d1.text((xpos, ypos), text, fill =(255, 255, 255), font=font)
# show image
img.show()


In [6]:
import os.path

def create_image(string, font='arial.ttf', show=False):
    if not os.path.isfile(f'./images/{string}.jpeg'):
        img = Image.new('RGB', (1024, 128))
        text = string
        # use declared font
        font = ImageFont.truetype(f"./fonts/{font}",70)
        # draw image
        d1 = ImageDraw.Draw(img)
        # Center text in image
        xpos = (img.size[0] / 2) - (font.getsize(text)[0]/2)
        ypos = (img.size[1] / 2) - (font.getsize(text)[1]/2)
        d1.text((xpos, ypos), text, fill =(255, 255, 255), font=font)
        # show and save the image
        if show:
            img.show()
        img.save(f'images/{string}.jpeg')

# for test_string in [test_string1, test_string2]:
#     create_image(test_string, show=True)


### Simple

In [104]:
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import numpy as np
import cv2

def calculate_similarity(string_a, string_b):
    imageA = cv2.imread(f'./images/{string_a}.jpeg')
    imageB= cv2.imread(f'./images/{string_b}.jpeg')
    gsA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    gsB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    # Calculate the MSE, SSIM, COR
    m = mse(gsA, gsB)
    s = ssim(gsA, gsB)
    return  m, s
# calculate_similarity(test_string1, test_string2)

In [105]:
for index, row in twisted_df.iterrows():
    create_image(row['Domain'])
    create_image(row['Homograph'])
    m, s = calculate_similarity(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'IMG_MSE'] = m
    twisted_df.loc[index,'IMG_SSM'] = s
twisted_df


KeyboardInterrupt: 

In [12]:
twisted_df = pd.read_csv('twisted_viz_sim.csv', index_col=0)
twisted_df

,Homograph,Domain,Registered,Levenshtein,Jaro-Winkler,Sorenson-Dice,Cosine,MSE,SSM
0,google.com,google.com,True,0.0,0.000000,0.000000,1.000000,0.000000,1.000000
1,google7.com,google.com,True,1.0,0.013774,0.294118,0.843274,3573.467743,0.848874
2,googlea.com,google.com,True,1.0,0.013774,0.294118,0.843274,3562.883354,0.851078
3,googled.com,google.com,True,1.0,0.013774,0.294118,0.843274,3576.705872,0.849858
4,googlej.com,google.com,True,1.0,0.013774,0.294118,0.843274,3890.821388,0.849594
...,...,...,...,...,...,...,...,...,...
47009,i.nstagram.com,instagram.com,False,1.0,0.022109,0.130435,0.880705,4724.995277,0.821886
47010,in.stagram.com,instagram.com,False,1.0,0.020408,0.217391,0.880705,4730.435272,0.820904
47011,inst.agram.com,instagram.com,False,1.0,0.071952,0.217391,0.880705,4727.817451,0.821145
47012,insta.gram.com,instagram.com,False,1.0,0.048273,0.217391,0.880705,4748.178596,0.821017


### Deep Learning

In [11]:
import torch
from torchvision import models, transforms

def siamese_similarity(string_a, string_b):
    # preprocess images
    imgA = Image.open(f'./images/{string_a}.jpeg').convert('RGB')
    imgB = Image.open(f'./images/{string_b}.jpeg').convert('RGB')

    transform = transforms.ToTensor()
    imgA = transform(imgA).unsqueeze(0)
    imgB = transform(imgB).unsqueeze(0)

    net = models.resnet50(weights='ResNet50_Weights.DEFAULT')
    embedding_size = net.fc.in_features
    net.fc = torch.nn.Linear(embedding_size, 256)

    embedding1 = net(imgA).detach()
    embedding2 = net(imgB).detach()
    e = torch.nn.functional.pairwise_distance(embedding1, embedding2).item()
    c = torch.nn.functional.cosine_similarity(embedding1, embedding2).item()
    l = torch.nn.functional.l1_loss(embedding1, embedding2).item()
    return e, c, l



In [13]:
for index, row in twisted_df.iterrows():
    create_image(row['Domain'])
    create_image(row['Homograph'])
    e, c, l = siamese_similarity(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'EMBD_EUC'] = e
    twisted_df.loc[index,'EMBD_COS'] = c
    twisted_df.loc[index,'EMBD_L1'] = l

twisted_df

,Homograph,Domain,Registered,Levenshtein,Jaro-Winkler,Sorenson-Dice,Cosine,MSE,SSM,EMBD_EUC,EMBD_COS,EMBD_L1
0,google.com,google.com,True,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,0.000016,1.000000,0.000000
1,google7.com,google.com,True,1.0,0.013774,0.294118,0.843274,3573.467743,0.848874,0.464163,0.801736,0.023069
2,googlea.com,google.com,True,1.0,0.013774,0.294118,0.843274,3562.883354,0.851078,0.456447,0.780336,0.022861
3,googled.com,google.com,True,1.0,0.013774,0.294118,0.843274,3576.705872,0.849858,0.523681,0.818360,0.026033
4,googlej.com,google.com,True,1.0,0.013774,0.294118,0.843274,3890.821388,0.849594,0.409549,0.863152,0.020503
...,...,...,...,...,...,...,...,...,...,...,...,...
47009,i.nstagram.com,instagram.com,False,1.0,0.022109,0.130435,0.880705,4724.995277,0.821886,0.568484,0.827922,0.029100
47010,in.stagram.com,instagram.com,False,1.0,0.020408,0.217391,0.880705,4730.435272,0.820904,0.730263,0.771949,0.035950
47011,inst.agram.com,instagram.com,False,1.0,0.071952,0.217391,0.880705,4727.817451,0.821145,0.609621,0.828346,0.030572
47012,insta.gram.com,instagram.com,False,1.0,0.048273,0.217391,0.880705,4748.178596,0.821017,0.690725,0.784406,0.034384


In [14]:
twisted_df.to_csv('twisted_viz_sim_siamese.csv')

### Single Value Classifiers

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import export_text, DecisionTreeClassifier
import numpy as np

In [103]:
# Functions for running and validating supervised classification models
def model_eval(df, target, model):

    kf = StratifiedKFold(n_splits=5, shuffle=True)

    # Arrays to store metrics
    recall = []
    precision = []
    f1 = []

    for train_index, test_index in kf.split(df, target):
        X_train, X_test = df.iloc[train_index], df.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        recall += [recall_score(y_pred, y_test, average="weighted", labels=[1], zero_division=0)]
        precision += [precision_score(y_pred, y_test, average="weighted", labels=[1])]
        f1 += [f1_score(y_pred, y_test, average="weighted", labels=[1])]

    print("precision = {:.4f} ±{:.4f}".format(np.mean(precision), np.std(precision)))
    print("recall    = {:.4f} ±{:.4f}".format(np.mean(recall), np.std(recall)))
    print("f1        = {:.4f} ±{:.4f}".format(np.mean(f1), np.std(f1)))
    return  X_test, y_test, model.predict_proba(X_test)

In [123]:
# Add comment
dt_model = DecisionTreeClassifier(random_state=42, max_depth=1)
features = twisted_df.set_index('Homograph').drop(['Domain', 'Registered'], axis=1).set_index(twisted_df['Homograph'])
for col in features.columns:
    print (col)
    model_eval(twisted_df[[col]], twisted_df['Registered'] , dt_model)
    print()
    print(export_text(dt_model, feature_names=[col], decimals=10))


Levenshtein
precision = 0.8036 ±0.0341
recall    = 0.6803 ±0.0236
f1        = 0.7359 ±0.0122

|--- Levenshtein <= 1.5000000000
|   |--- class: True
|--- Levenshtein >  1.5000000000
|   |--- class: False

Jaro-Winkler
precision = 0.8261 ±0.0191
recall    = 0.6429 ±0.0141
f1        = 0.7228 ±0.0058

|--- Jaro-Winkler <= 0.1100206599
|   |--- class: True
|--- Jaro-Winkler >  0.1100206599
|   |--- class: False

Sorenson-Dice
precision = 0.8125 ±0.0160
recall    = 0.6446 ±0.0121
f1        = 0.7187 ±0.0091

|--- Sorenson-Dice <= 0.3798076957
|   |--- class: True
|--- Sorenson-Dice >  0.3798076957
|   |--- class: False

Cosine
precision = 0.8292 ±0.0125
recall    = 0.6627 ±0.0147
f1        = 0.7365 ±0.0106

|--- Cosine <= 0.7476779819
|   |--- class: False
|--- Cosine >  0.7476779819
|   |--- class: True

MSE
precision = 0.6438 ±0.0042
recall    = 0.5163 ±0.0232
f1        = 0.5728 ±0.0154

|--- MSE <= 3934.4249267578
|   |--- class: True
|--- MSE >  3934.4249267578
|   |--- class: False

SSM


### Multi Value Clasifiers

In [124]:
features

,Levenshtein,Jaro-Winkler,Sorenson-Dice,Cosine,MSE,SSM,EMBD_EUC,EMBD_COS,EMBD_L1,Negative Prediction,Positive Prediction
Homograph,,,,,,,,,,,
google.com,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,0.000016,1.000000,0.000000,0.83,0.17
google7.com,1.0,0.013774,0.294118,0.843274,3573.467743,0.848874,0.464163,0.801736,0.023069,0.00,1.00
googlea.com,1.0,0.013774,0.294118,0.843274,3562.883354,0.851078,0.456447,0.780336,0.022861,0.00,1.00
googled.com,1.0,0.013774,0.294118,0.843274,3576.705872,0.849858,0.523681,0.818360,0.026033,0.00,1.00
googlej.com,1.0,0.013774,0.294118,0.843274,3890.821388,0.849594,0.409549,0.863152,0.020503,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...
i.nstagram.com,1.0,0.022109,0.130435,0.880705,4724.995277,0.821886,0.568484,0.827922,0.029100,0.00,1.00
in.stagram.com,1.0,0.020408,0.217391,0.880705,4730.435272,0.820904,0.730263,0.771949,0.035950,0.00,1.00
inst.agram.com,1.0,0.071952,0.217391,0.880705,4727.817451,0.821145,0.609621,0.828346,0.030572,0.00,1.00


In [119]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42, criterion="entropy")
features = twisted_df.drop(['Domain', 'Homograph', 'Registered'], axis=1)
model_eval(features[['MSE', 'SSM']], twisted_df['Registered'] , rf_model)

   

precision = 0.6354 ±0.0214
recall    = 0.7375 ±0.0208
f1        = 0.6823 ±0.0148


(               MSE       SSM
 5      3535.652359  0.851115
 7      3533.812370  0.850705
 9      3475.654427  0.852781
 16     3597.494606  0.847997
 20     3542.201500  0.849967
 ...            ...       ...
 46992  4088.740196  0.827028
 47001    54.547775  0.995567
 47006   340.211365  0.984923
 47009  4724.995277  0.821886
 47011  4727.817451  0.821145
 
 [9402 rows x 2 columns],
 5         True
 7         True
 9         True
 16        True
 20        True
          ...  
 46992    False
 47001    False
 47006    False
 47009    False
 47011    False
 Name: Registered, Length: 9402, dtype: bool,
 array([[0.13, 0.87],
        [0.  , 1.  ],
        [0.19, 0.81],
        ...,
        [0.98, 0.02],
        [0.18, 0.82],
        [0.3 , 0.7 ]]))

In [38]:
rf_model = RandomForestClassifier(random_state=42)
features = twisted_df.drop(['Domain', 'Homograph', 'Registered'], axis=1)
model_eval(features[['EMBD_EUC','EMBD_L1']], twisted_df['Registered'] , rf_model)

precision = 0.3195 ±0.0188
recall    = 0.5942 ±0.0165
f1        = 0.4152 ±0.0173


In [95]:
rf_model = RandomForestClassifier(random_state=42)
features = twisted_df.drop(['Domain', 'Homograph', 'Registered'], axis=1)
model_eval(features[['SSM']], twisted_df['Registered'] , rf_model)

precision = 0.4924 ±0.0250
recall    = 0.4968 ±0.0173
f1        = 0.4946 ±0.0211


array([[0.  , 1.  ],
       [0.69, 0.31],
       [0.18, 0.82],
       ...,
       [1.  , 0.  ],
       [0.02, 0.98],
       [0.96, 0.04]])

In [125]:
lev_df, lev_class, probabilities = model_eval(features[['SSM', 'MSE']], twisted_df['Registered'], rf_model)
neg = [i[0] for i in probabilities]
pos = [i[1] for i in probabilities]
print(neg)
print (pos)

precision = 0.6333 ±0.0169
recall    = 0.7444 ±0.0117
f1        = 0.6843 ±0.0136
[0.0, 0.02, 0.03, 0.13, 0.24, 0.01, 0.35, 0.01, 0.02, 0.01, 0.01, 0.35, 0.01, 0.27, 0.03, 0.95, 0.96, 0.93, 0.69, 0.97, 0.9, 0.94, 0.14, 0.36, 0.21, 1.0, 0.02, 0.01, 0.95, 1.0, 0.98, 0.04, 0.0, 1.0, 0.99, 0.77, 0.73, 1.0, 0.01, 0.01, 0.38, 0.31, 0.08, 0.15, 0.18, 0.09, 0.94, 0.11, 0.03, 0.0, 0.02, 0.12, 0.08, 0.08, 0.3, 0.73, 0.49, 0.29, 1.0, 1.0, 1.0, 0.99, 1.0, 1.0, 1.0, 0.98, 1.0, 0.78, 0.92, 0.99, 0.99, 1.0, 1.0, 0.94, 1.0, 0.99, 0.99, 0.96, 0.67, 0.93, 1.0, 1.0, 0.88, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 1.0, 1.0, 0.7, 1.0, 1.0, 1.0, 0.97, 1.0, 0.9, 0.97, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.98, 0.99, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.91, 0.79, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.91, 0.99, 0.83, 1.0, 0.99, 0.92, 0.71, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.67, 1.0, 1.0, 1.0, 0.78, 1.0, 1.0, 0.84, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [126]:
lev_df['Registered'] = lev_class.values
lev_df['Negative Prediction'] = neg
lev_df['Positive Prediction'] = pos
lev_df

,SSM,MSE,Registered,Negative Prediction,Positive Prediction
Homograph,,,,,
google.com,1.000000,0.000000,True,0.00,1.00
googlea.com,0.851078,3562.883354,True,0.02,0.98
googlej.com,0.849594,3890.821388,True,0.03,0.97
googlee.com,0.851325,3526.249039,True,0.13,0.87
googleb.com,0.849967,3542.201500,True,0.24,0.76
...,...,...,...,...,...
instfagram.com,0.818099,4846.953651,False,0.23,0.77
insytagram.com,0.831153,3826.378334,False,0.91,0.09
inystagram.com,0.832464,3782.782341,False,0.85,0.15


In [128]:
# Create boolean mask based on the conditions
mask = (lev_df['Registered'] & (lev_df['Positive Prediction'] > lev_df['Negative Prediction']))

# Filter the DataFrame based on the mask
filtered_df = lev_df.loc[mask]

# View the filtered DataFrame
filtered_df

,SSM,MSE,Registered,Negative Prediction,Positive Prediction
Homograph,,,,,
google.com,1.000000,0.000000,True,0.00,1.00
googlea.com,0.851078,3562.883354,True,0.02,0.98
googlej.com,0.849594,3890.821388,True,0.03,0.97
googlee.com,0.851325,3526.249039,True,0.13,0.87
googleb.com,0.849967,3542.201500,True,0.24,0.76
...,...,...,...,...,...
inatagram.com,0.909263,1269.008804,True,0.36,0.64
insragram.com,0.908911,1272.707367,True,0.26,0.74
isntagram.com,0.978598,451.282753,True,0.37,0.63


In [129]:
sorted_df = filtered_df.sort_values(by='Positive Prediction', ascending=False)
sorted_df

,SSM,MSE,Registered,Negative Prediction,Positive Prediction
Homograph,,,,,
google.com,1.000000,0.000000,True,0.00,1.00
qoogle.com,0.993889,98.837692,True,0.00,1.00
facebook.com,1.000000,0.000000,True,0.00,1.00
facebonk.com,0.991980,132.906586,True,0.00,1.00
tacebook.com,0.995574,54.321037,True,0.00,1.00
...,...,...,...,...,...
microwoft.com,0.834308,4384.893112,True,0.47,0.53
iicrosoft.com,0.862836,3179.951111,True,0.47,0.53
facebopok.com,0.816222,4293.016808,True,0.47,0.53


In [140]:
import Levenshtein

for i in list(sorted_df.index):
    output =  (Levenshtein.editops("google.com", i))
    p = []
    for item in output:
        p.append([item[0], i[item[1]],  i[item[2]]])
    print (p)


[]
[['replace', 'q', 'q']]
[['insert', 'f', 'f'], ['insert', 'f', 'a'], ['replace', 'f', 'c'], ['replace', 'a', 'e'], ['replace', 'c', 'b'], ['replace', 'e', 'o'], ['replace', 'b', 'o'], ['replace', 'o', 'k']]
[['insert', 'f', 'f'], ['insert', 'f', 'a'], ['replace', 'f', 'c'], ['replace', 'a', 'e'], ['replace', 'c', 'b'], ['replace', 'e', 'o'], ['replace', 'b', 'n'], ['replace', 'o', 'k']]
[['insert', 't', 't'], ['insert', 't', 'a'], ['replace', 't', 'c'], ['replace', 'a', 'e'], ['replace', 'c', 'b'], ['replace', 'e', 'o'], ['replace', 'b', 'o'], ['replace', 'o', 'k']]
[['insert', 'a', 'a'], ['insert', 'a', '-'], ['replace', 'a', 'm'], ['replace', '-', 's'], ['replace', 'm', 'e'], ['replace', 's', 'd'], ['replace', 'e', 'g'], ['replace', 'e', 'n'], ['replace', '.', 'e'], ['replace', 'n', 't']]
[['replace', 'y', 'y'], ['insert', 'u', 'u'], ['replace', 'u', 't'], ['replace', 't', '5'], ['replace', '5', 'b']]
[['replace', 'y', 'y'], ['insert', 'u', 'u'], ['replace', 'u', 't'], ['replace',